<img src="https://www.haskell.org/static/img/haskell-logo.svg?etag=ukf3Fg7-">

# Yu Gai

# Purity and Monad

Everything that cannot ensure referential transparency is impure.

Examples:

- IO

- Concurrency (shared variable, channel, etc.)

Well, most meaningful applications cannot be programmed "purely".

In fact, impure values exist in Haskell, referred to as **actions**.

In [1]:
import System.IO

:t putStrLn -- "print v" is equivalent to "putStrLn (show v)" (Remember "deriving Show"?)
:t getLine
:t hPutStrLn
:t hGetLine

putStrLn :: String -> IO ()

getLine :: IO String

hPutStrLn :: Handle -> String -> IO ()

hGetLine :: Handle -> IO String

In [2]:
:t withFile

withFile :: forall r. FilePath -> IOMode -> (Handle -> IO r) -> IO r

Does this remind you of parameterized types?

There is nothing magical! "IO r" is simply a parameterized type.

But how to deconstruct it? (In Haskell, deconstruction refers to the retrieval of value used for construction)

In [3]:
get h = do
    ln <- hGetLine h -- bind a pattern to the result of an action (can be thought of as deconstructing "IO string")
    putStrLn ln
withFile "README.md" ReadMode get

IHaskell is a kernel for the [Jupyter project](http://ipython.org), which allows you to use Haskell inside Jupyter frontends, including the console and in-browser notebook.

In [4]:
get h = do
    ln <- hGetLine h
    -- attempting to return ln as a pure expression (ln becomes pure after deconstruction)
    ln
withFile "README.md" ReadMode get

Line 1: Use join
Found:
do ln <- hGetLine h
   ln
Why not:
do join (hGetLine h)

Couldn't match type `[]' with `IO'
Expected type: IO Char
  Actual type: String
In a stmt of a 'do' block: ln
In the expression:
  do { ln <- hGetLine h;
       ln }

In [5]:
get h = do -- stupid example
    ln <- hGetLine h
    return ln
withFile "README.md" ReadMode get

Line 1: Redundant return
Found:
do ln <- hGetLine h
   return ln
Why not:
do hGetLine h

"IHaskell is a kernel for the [Jupyter project](http://ipython.org), which allows you to use Haskell inside Jupyter frontends, including the console and in-browser notebook."

In [6]:
-- return is a function rather than a control-flow statement

:t return -- Why "Monad m"? To be explained later.

return :: forall (m :: * -> *) a. Monad m => a -> m a

In [7]:
-- >>= is the equivalence of . operator in do-block (stupid example)

get h = do
    hGetLine h >>= putStrLn
withFile "README.md" ReadMode get

Line 1: Redundant do
Found:
do hGetLine h >>= putStrLn
Why not:
hGetLine h >>= putStrLn

IHaskell is a kernel for the [Jupyter project](http://ipython.org), which allows you to use Haskell inside Jupyter frontends, including the console and in-browser notebook.

In [8]:
-- A do-block is in fact the syntactic sugar for >>=.

get h = hGetLine h >>= \s -> putStrLn s
withFile "README.md" ReadMode get

IHaskell is a kernel for the [Jupyter project](http://ipython.org), which allows you to use Haskell inside Jupyter frontends, including the console and in-browser notebook.

In [9]:
import Control.Concurrent.MVar

:t newMVar
:t takeMVar
:t putMVar

newMVar :: forall a. a -> IO (MVar a)

takeMVar :: forall a. MVar a -> IO a

putMVar :: forall a. MVar a -> a -> IO ()

In [10]:
-- a question encountered previously

:t return -- Why "Monad m"?

return :: forall (m :: * -> *) a. Monad m => a -> m a

IO as a parameterized type is an "instance" of the Monad "class".

The "return" function and >>= operator are methods of the Monad class.

Consequence: >>= and "return" can be generalized to more "instances"!

In [16]:
-- An example: the Maybe monad.

import Data.Maybe

divide x 0 = Nothing
divide x y = return (x / y)

-- How to avoid divide-by-zero in "f x y = x / y + y / x"?

f x y = do
    -- If "divide x y" yield "Nothing", "f x y" will yield "Nothing" without evaluating consequtive expressions. Why?
    xy <- divide x y
    yx <- divide y x
    return (xy + yx)
    
f 1 0

Nothing

In [ ]:
-- Reason: (this is ROUGHLY how Maybe is implemented)

data Maybe t = Just t | Nothing

instance Monad Maybe  where
    (Just x) >>= k = k x
    Nothing >>= _  = Nothing
    return = Just
    fail _ = Nothing